# 要約 
このJupyter Notebookは、LMSYS - Chatbot Arenaコンペティションにおける人間による好み予測のための深層学習モデルを構築・評価するためのコードを提供しています。以下にその要約を示します。

### 問題設定
Notebookは、人間の好みを予測するためのモデルを構築するという課題に取り組んでいます。具体的には、二つの異なるチャットボット（モデル）からの応答のうち、どちらがユーザーに好まれるかを予測することが目標です。このプロセスでは、使用するモデルのトークン化やスコアリングを行い、最終的に予測結果を生成します。

### 使用する手法とライブラリ
1. **ライブラリのインストール**:
   - `transformers`: 大規模言語モデル（LLM）を扱うライブラリ。
   - `peft`, `bitsandbytes`, `accelerate`: モデルの最適化や量子化に役立つライブラリ。
   - `einops`: テンソルの操作を簡素化するライブラリ。
   - `torch`: PyTorchライブラリを使用し、深層学習フレームワークとしての基盤を提供。

2. **モデルの準備**:
   - 複数の事前トレーニング済みモデル（1.8B、2.7B、20B）のロードと量子化を行い、GPUに配置します。
   - `BitsAndBytesConfig`を使用して、モデルのメモリ効率を向上させる4ビット量子化を実施。

3. **データの前処理**:
   - CSV形式のテストデータを読み込み、プロンプトと応答の長さを最大7200トークンに制限するカットオフ処理を行います。
   - 各データ例に対して、プロンプトと応答の整形を実施。

4. **推論の実行**:
   - モデルを使って推論し、各応答に対するスコアを計算します。スコアの差が小さい場合は引き分けとし、それ以外の場合は勝者モデルのスコアを確率に変換します。
   - 推論関数を2つのスレッドで並列実行し、効率的に処理を行います。

5. **予測結果の保存**:
   - 最終的な予測結果をDataFrameにまとめ、`submission.csv`として保存します。また、エラーカウントや予測性能評価（対数損失など）も行う準備があります。

### 結論
このNotebookは、効果的に人間の好みを予測するための一連の処理を構築し、特定のチャットボットの応答の好まれ方を理解するための基盤を提供します。使用されている技術的な手法やライブラリは、最新の深層学習モデルを精細に調整し、評価するためのものです。利用者は、最終的に生成された`submission.csv`ファイルを用いてコンペティションに参加することが可能です。

---


# 用語概説 
以下に、初心者がつまずきそうな専門用語に関する簡単な解説を列挙します。特に、マイナーなものや実務経験のない人に馴染みがない可能性のある用語に焦点を当てています。

### 専門用語の解説

1. **transformers**:
   - 自然言語処理（NLP）モデルのアーキテクチャであり、特に注意機構に基づくモデル群を指します。BERTやGPTシリーズなど、最近のさまざまなLLMがこのアーキテクチャに基づいています。

2. **peft (Parameter-Efficient Fine-Tuning)**:
   - 設計したモデルの大部分のパラメータを固定し、必要な部分だけを調整する方法です。これは大規模モデルの微調整時に必要な計算リソースを削減します。

3. **bitsandbytes**:
   - モデルのエフィシエンシーやメモリ使用効率を向上させるためのライブラリで、量子化技術を利用してモデルのサイズを小さくすることができます。

4. **accelerate**:
   - PyTorchを利用した分散トレーニングの短縮化や効率化のためのライブラリです。複数のGPUやTPUを簡単に活用するためのツールを提供します。

5. **量子化 (Quantization)**:
   - モデルの重みとアクティベーションを低ビットの整数で表現する手段です。これにより、モデルのメモリ使用量が減少し、推論を高速化できます。

6. **ダブル量子化 (Double Quantization)**:
   - 量子化を2回行うことで、モデルの精度を維持しながらさらなるメモリ削減を図る技術です。一度目の量子化後に再度量子化を行います。

7. **tensors**:
   - NumPyの配列に似た多次元のデータ構造で、PyTorchやTensorFlowなどで計算やデータの格納に使用されます。特に深層学習では重要な役割を果たします。

8. **プロンプト (Prompt)**:
   - 言語モデルに与えられる入力テキストのことです。モデルはこのプロンプトに基づいて応答を生成します。

9. **エラーカウント (Error Count)**:
   - モデルが推論中に遭遇したエラーの数をカウントすることです。これにより、推論の安定性を評価することができます。

10. **&delta; (デルタ)**:
    - 数学や統計で用いられる差異を示すシンボル。特にスコアなどの変化を表すのに使われます。

11. **IPython.display**:
    - Jupyter Notebook内でのオブジェクトの表示を管理するためのモジュールです。ここでは、ファイルへのリンクを作成するために用いられています。

これらの用語は、特にこのノートブックでのプロセスや実装に関連しており、初心者にとっては実務上現れることの少ないものが多いです。理解を深めるために、それぞれの用語についてさらに調査することをお勧めします。

---


In [ ]:
# transformers、peft、bitsandbytes、accelerateのライブラリを最新バージョンにアップデートしてインストールします。
# --no-indexオプションを使用して、PyPIから直接ではなく、指定したリンクからインストールします。
!pip install -U transformers peft bitsandbytes accelerate --no-index --find-links /kaggle/input/lmsys-wheel-files

# einopsライブラリを最新バージョンにアップデートしてインストールします。
# 同様に、指定されたリンクからインストールします。
!pip install -q -U einops --no-index --find-links /kaggle/input/einops-v0-8-0

In [ ]:
# PyTorchライブラリをインポートします。これは深層学習フレームワークです。
import torch

# AutoModelおよびAutoTokenizerクラスをtransformersライブラリからインポートします。
# AutoModelは、指定されたモデル名に基づいて適切なモデルを自動的にロードします。
# AutoTokenizerは、入力テキストをトークンに変換するためのツールです。
from transformers import AutoModel, AutoTokenizer

# BitsAndBytesConfigクラスをtransformersライブラリからインポートします。
# これは、モデルの量子化やメモリ効率の向上に関連する設定を扱います。
from transformers import BitsAndBytesConfig

In [ ]:
# 使用するモデルのパスを指定します。
# 最初は1.8Bモデル、次に2.7Bモデル、最後に20Bモデルのパスが示されています。
model_path = "/kaggle/input/internlm2-1.8b-reward/transformers/default/1/internlm_internlm2-1_8b-reward"
model_path = "/kaggle/input/internlm-2-7b/transformers/default/1/internlm_internlm2-7b-reward"
model_path = "/kaggle/input/iternlm2-20b-reward/transformers/default/1/internlm_internlm2-20b-reward"

# 指定したモデルパスからトークナイザーをロードします。
# trust_remote_code=Trueは、モデル構成のリモートコードを信頼する設定です。
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# BitsAndBytesConfigの設定を作成します。
# load_in_4bit=Trueは、4ビットモデルのロードを有効にします。
# bnb_4bit_quant_typeで量子化のタイプを指定します。
# bnb_4bit_use_double_quant=Trueは、ダブル量子化を使用するかどうかを示します。
# bnb_4bit_compute_dtypeは、計算時に使用するデータ型を指定します。
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# model_0として指定したパスから事前トレーニング済みのモデルをロードします。
# device_map="cuda:0"は、最初のGPUデバイスを指定しています。
# torch_dtype=torch.float16は、モデルが使用するデータ型を指定しています。
model_0 = AutoModel.from_pretrained(
    model_path, 
    device_map="cuda:0", 
    torch_dtype=torch.float16, 
    trust_remote_code=True,
    quantization_config=bnb_config,
)

# model_1として同様に指定したパスから事前トレーニング済みのモデルをロードします。
# device_map="cuda:1"は、2番目のGPUデバイスを指定しています。
model_1 = AutoModel.from_pretrained(
    model_path, 
    device_map="cuda:1", 
    torch_dtype=torch.float16, 
    trust_remote_code=True,
    quantization_config=bnb_config,
)

# 次に、同じモデルをGPUデバイス2000に向けてタブ使用でロードするコードのコメントアウトされた部分があります。
# コメントアウトされているため、実行されませんが、同様の意味を持っています。
# model_0 = AutoModel.from_pretrained(
#     model_path, 
#     device_map="cuda:0", 
#     torch_dtype=torch.float16, 
#     trust_remote_code=True,
# )
# model_1 = AutoModel.from_pretrained(
#     model_path, 
#     device_map="cuda:1", 
#     torch_dtype=torch.float16, 
#     trust_remote_code=True,
# )

In [ ]:
# 4ビット量子化されたモデルの保存先パスを指定します。
# ここでは"internlm2-20b-rm-bnb-4bit"という名前です。
# bnb_4bit_path = "internlm2-20b-rm-bnb-4bit"

# model_0（最初のモデル）を指定したパスに保存します。
# save_pretrainedメソッドを使用して、モデルの状態をディスクに保存します。
# model_0.save_pretrained(bnb_4bit_path)

# tokenizer（トークナイザー）を指定したパスに保存します。
# これにより、このトークナイザーの設定や辞書などが保存されます。
# tokenizer.save_pretrained(bnb_4bit_path)

In [ ]:
# IPython.displayライブラリからFileLinkとdisplay関数をインポートします。
# FileLinkは、ファイルへのリンクを作成するためのツールです。
# display関数は、与えられたオブジェクトを表示します。
# from IPython.display import FileLink, display

# 指定されたファイルへのリンクを作成し、そのリンクを表示します。
# ファイルのパスは"/kaggle/working/internlm2-20b-rm-bnb-4bit/model-00001-of-00003.safetensors"です。
# display(FileLink("/kaggle/working/internlm2-20b-rm-bnb-4bit/model-00001-of-00003.safetensors"))

In [ ]:
# model_0の内容を表示します。
# これにより、モデルの情報や設定、構造などの詳細が表示されます。
model_0

In [ ]:
# pandasライブラリをインポートします。
# pandasはデータ解析用の強力なツールです。
import pandas as pd

# DEBUGフラグを初期化します。このフラグは、デバッグモードを示します。
# DEBUG = False

# test.csvファイルを読み込み、DataFrameとしてdfに格納します。
df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# DataFrameの行数が3の場合、デバッグモードを有効にしてトレーニング用のデータセットを読み込むようにします。
# デバッグモードであれば、train.csvファイルを読み込み、最初の1000行のみを取得します。
# if len(df) == 3:
#     DEBUG = True
#     df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
#     df = df.head(1000)

# 読み込んだDataFrame dfを表示します。
df

In [ ]:
# 与えられたデータ例のプロンプトと応答を処理する関数を定義します。
# この関数は、テキストの長さが指定された最大値を超えないようにカットします。
def cut_off(example, max_length=7200):
    
    # 指定されたインデックスまでのテキストの長さをカウントする内部関数を定義します。
    def _count(example, idx):
        _len = 0    
        # プロンプトと両方の応答の長さを合計します。
        for s in example['prompt'][:idx] + example['response_a'][:idx] + example['response_b'][:idx]:
            _len += len(s)
        return _len

    # 再帰的にカットを実行する内部関数を定義します。
    def _recusive_cut(idx):
        if idx == 0:
            # インデックスが0の場合、最大の長さに切り捨てます。
            example['prompt'] = [example['prompt'][0][:1000]]
            example['response_a'] = [example['response_a'][0][:3000]]
            example['response_b'] = [example['response_b'][0][:3000]]
            return example
            
        # 指定したインデックスまでの長さが最大値を超えているか確認します。
        if _count(example, idx) > max_length:
            # 超えている場合、インデックスを1つ減らして再帰的に実行します。
            return _recusive_cut(idx-1)
        else:
            # 最大値を超えていない場合、プロンプトと応答を適切なインデックスまで切り捨てます。
            example['prompt'] = example['prompt'][:idx]
            example['response_a'] = example['response_a'][:idx]
            example['response_b'] = example['response_b'][:idx]
            return example
    # プロンプトの長さに基づいて再帰的にカットを実行します。
    return _recusive_cut(len(example['prompt']))


# 例を処理するための関数を定義します。
def process_fn(example):
    # evalを使用して、文字列をPythonのオブジェクトに変換します。
    example['prompt'] = eval(example['prompt'], {"null": ""})
    example['response_a'] = eval(example['response_a'], {"null": ""})
    example['response_b'] = eval(example['response_b'], {"null": ""})
    # カットオフを実行します。
    return cut_off(example)

# DataFrame dfの各行に対してprocess_fnを適用し、新しいDataFrame new_dfを作成します。
new_df = df.apply(lambda x: process_fn(x), axis=1)

# 新しく生成されたDataFrame new_dfを表示します。
new_df

In [ ]:
# コードの実行時間を計測するためのマジックコマンドです。
%%time

# mathライブラリとnumpyライブラリをインポートします。
import math
import numpy as np

# データフレームとモデルを引数に取り、推論を行う関数を定義します。
def inference(df, model):

    error_cnt = 0  # エラーのカウントを初期化します。
    y_pred = []  # 予測結果を保存するリストを初期化します。
    
    # DataFrameの各行を繰り返します。
    for idx, row in df.iterrows():
        chat_a = []  # モデルA用のチャットデータを初期化します。
        chat_b = []  # モデルB用のチャットデータを初期化します。
        
        # 各プロンプトに対して応答を追加します。
        for i in range(len(row['prompt'])):
            chat_a.append({"role": "user", "content": row['prompt'][i]})  # ユーザーのプロンプトを追加します。
            chat_a.append({"role": "assistant", "content": row['response_a'][i]})  # モデルAの応答を追加します。

            chat_b.append({"role": "user", "content": row['prompt'][i]})  # ユーザーのプロンプトを追加します。
            chat_b.append({"role": "assistant", "content": row['response_b'][i]})  # モデルBの応答を追加します。
            
        # モデルを使ってスコアを取得します。
        try:
            score1, score2 = model.get_scores(tokenizer, [chat_a, chat_b])  # モデルからスコアを取得します。
            
            # スコアの差が0.08未満の場合は、両モデルの勝ちをほぼ同じとみなします。
            if abs(score1 - score2) < 0.08:
                y_pred.append([0.00005, 0.00005, 0.9999])  # 引き分けの場合の予測を追加します。
            else:
                # スコアを指数関数的に変換し、確率を計算します。
                score1, score2 = math.exp(score1), math.exp(score2)
                sum_ = score1 + score2  # スコアの合計を計算します。
                y_pred.append([score1/sum_ - 0.0001, score2/sum_ - 0.0001, 0.0002])  # 確率をリストに追加します。
        except:
            # エラーが発生した場合は、均等な確率を割り当てます。
            y_pred.append([0.33334, 0.33333, 0.33333])
            error_cnt += 1  # エラーカウントを増やします。
        
    # 予測結果をNumPy配列に変換します。
    y_pred = np.array(y_pred)
    
    # DataFrameに各モデルの予測結果を追加します。
    df['winner_model_a_pred'] = y_pred[:, 0]  # モデルAの勝者予測を追加します。
    df['winner_model_b_pred'] = y_pred[:, 1]  # モデルBの勝者予測を追加します。
    df['winner_tie_pred'] = y_pred[:, 2]  # 引き分け予測を追加します。
    
    print(error_cnt)  # エラーの数を表示します。
    return df  # 予測結果を含むDataFrameを返します。

In [ ]:
# 推論関数をテストします。
# new_dfの最初の20行を用いて、model_0を使った推論を実行します。
# inference(new_df[:20], model_0)

In [ ]:
# コードの実行時間を計測するためのマジックコマンドです。
%%time

# new_dfの偶数番目の行を選択し、コピーを作成します。これがsub_0になります。
sub_0 = new_df.iloc[0::2].copy()
# new_dfの奇数番目の行を選択し、コピーを作成します。これがsub_1になります。
sub_1 = new_df.iloc[1::2].copy()

# concurrent.futuresからThreadPoolExecutorをインポートします。
from concurrent.futures import ThreadPoolExecutor 

# ThreadPoolExecutorを使用して、2つのスレッドで推論を並列実行します。
with ThreadPoolExecutor(max_workers=2) as executor:
    # inference関数をそれぞれのサブDataFrameとモデルに対して実行します。
    results = executor.map(inference, (sub_0, sub_1), (model_0, model_1))

# 結果をDataFrameに結合し、新しいDataFrame result_dfを作成します。
result_df = pd.concat(list(results), axis=0)

# 結果DataFrameの最初の5行を表示します。
result_df.head()

In [ ]:
# result_dfの列名を変更します。
# 'winner_model_a_pred'を'winner_model_a'に、
# 'winner_model_b_pred'を'winner_model_b'に、
# 'winner_tie_pred'を'winner_tie'に変更します。
result_df = result_df.rename(columns={'winner_model_a_pred' : 'winner_model_a', 'winner_model_b_pred' : 'winner_model_b', 'winner_tie_pred' : 'winner_tie'})

# 列名を変更した結果のDataFrame result_dfを表示します。
result_df

In [ ]:
# result_dfから'id', 'winner_model_a', 'winner_model_b', 'winner_tie'の列を選択し、'submission.csv'ファイルに書き出します。
# index=Falseは、行番号をCSVに含めないようにするオプションです。
result_df[['id', 'winner_model_a', 'winner_model_b', 'winner_tie']].to_csv('submission.csv', index=False)

# 書き出した'submission.csv'ファイルを読み込み、最初の5行を表示します。
pd.read_csv('submission.csv').head(5)

In [ ]:
# sklearn.metricsからlog_lossとaccuracy_scoreをインポートします。
# from sklearn.metrics import log_loss, accuracy_score

# デバッグモードが有効な場合に以下の処理を実行します。
# if DEBUG:
#     # 実際の結果を選択し、y_trueとしてリストに変換します。
#     y_true = result_df[['winner_model_a', 'winner_model_b', 'winner_tie']].values.tolist()
#     # 予測結果を選択し、y_predとしてリストに変換します。
#     y_pred = result_df[['winner_model_a_pred', 'winner_model_b_pred', 'winner_tie_pred']].values.tolist()
#     # log_lossを計算し、表示します。
#     print(log_loss(y_true, y_pred))

In [ ]:
# デバッグモードが無効な場合に以下の処理を実行します。
# if not DEBUG:
#     # result_dfの列名を再度変更します。
#     result_df = result_df.rename({'winner_model_a_pred' : 'winner_model_a', 'winner_model_b_pred' : 'winner_model_b', 'winner_tie_pred' : 'winner_tie'})
#     # 'id', 'winner_model_a', 'winner_model_b', 'winner_tie'の列を選択し、'submission.csv'ファイルに書き出します。
#     result_df[['id', 'winner_model_a', 'winner_model_b', 'winner_tie']].to_csv('submission.csv', index=False)
#     # 書き出した'submission.csv'ファイルを読み込み、最初の5行を表示します。
#     pd.read_csv('submission.csv').head(5)

---

# コメント 

> ## Dlond Mike
> 
> 8u nb lol:)

---
